In [ ]:
!pip install numpy pandas plotly scipy scikit-learn seaborn statsmodels

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/vzuev/Documents/work/phd/eeg/events/2025_sirius/homework/stats/dataframe_for_homework.csv", sep=";")
df.head()

Посмотрим на данные в целом. Сгруппируем по полу

In [ ]:
import seaborn as sns
sns.pairplot(df, hue="sex")

Визуально кажется, что `sd_corr` и `env_corr` не коррелируют (по крайней мере, попарно) с `age` и `PLS_receptive`.

Проверим коэффициенты корреляции Пирсона.
Не забываем сделать поправку на множественное тестирование (я выбрал "мягкий" метод Беньямини-Хохберга и уровень значимости alpha=0.01)

In [ ]:
from itertools import product
from scipy import stats
from statsmodels.stats.multitest import multipletests

pairs = list(product(("sd_corr", "env_corr"), ("age", "PLS_receptive")))
print("pairs:", pairs)

corr_results = []
for col1, col2 in pairs:
    r, p = stats.pearsonr(df[col1], df[col2])
    corr_results.append({'pair': f"{col1}-{col2}", 'r': r, 'p': p})
results_df = pd.DataFrame(corr_results)

rejected, p_adj, _, _ = multipletests(results_df['p'], method='fdr_bh', alpha=.01)
results_df['p_adj_bh'] = p_adj
results_df['significant_bh'] = rejected
results_df

Гипотеза H<sub>0</sub> для каждой пары величин - эти две величины независимые нормально распределённые.
p-value при проверки такой гипотезы, грубо говоря, соответствует вероятности выборки из некоррелированной системы двух величин иметь коэффициент корреляции R по модулю не меньше полученного при вычислении.

Получается, всё же параметры значимо коррелируют (кроме пары `PLS_receptive` и `env_corr`), поскольку скорректированные p-value < alpha.
Т.о. **нейронное отслеживание связано как с возрастом, так и со способностью понимать естественную речь**.

Построим линейную регрессию каждого коэффициента, используя возраст и уровень развития рецептивной речи как факторы

In [ ]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

model = LinearRegression()

x_col, y_col = "age", "PLS_receptive"
x_range = np.linspace(df[x_col].min(), df[x_col].max(), 20)
y_range = np.linspace(df[y_col].min(), df[y_col].max(), 20)
xx, yy = np.meshgrid(x_range, y_range)

def fit_and_plot(z_col: str) -> None:
    model.fit(df[[x_col, y_col]], df[z_col])
    zz = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
    fig = px.scatter_3d(df, x="age", y="PLS_receptive", z="env_corr", color="sex")
    
    go_fig = go.Figure(fig)
    go_fig.add_trace(go.Surface(x=xx, y=yy, z=zz, opacity=.6, showscale=False))
    # go_fig.show()
    go_fig.write_html(f"{z_col}_linearRegr.html")

    r_squared = model.score(df[[x_col, y_col]], df[z_col])
    print(f"R^2 = {r_squared:.3f}")

fit_and_plot(z_col="env_corr")

In [ ]:
fit_and_plot(z_col="sd_corr")

Коэффициент детерминации R<sup>2</sup> (доля объяснённой дисперсии зависимой переменной) невысока, но для `sd_corr`, пожалуй, даже выше, чем я ожидал по визуальной оценке. 

Окна Plotly не отображаются интерактивно при экспорте Jupyter в HTML, поэтому результаты приведены в отдельных HTML-файлах.